Importing Libraries

In [1]:
import cv2
import os 
from cvzone.HandTrackingModule import HandDetector
import numpy as np
import pyautogui
import time


Main Code 

In [3]:
# calculating width and height of screen
width, height = pyautogui.size()
w = []

cap = cv2.VideoCapture(0)
cap.set(3, width)
cap.set(4, height)

# initialize HandDetector to detect 1 hand with 80% accuracy
detector = HandDetector(detectionCon=0.8, maxHands=1)

dragging = False  # Variable to track dragging state

while True:
    # reading image
    success, img = cap.read()
    img = cv2.flip(img, 1)
    
    hands, img = detector.findHands(img)
    if hands:
        hand = hands[0]
        fingers = detector.fingersUp(hand)
        lmlist = hand['lmList']
        
        # Map the hand's position to the screen
        xVal = int(np.interp(lmlist[8][0], [0, width], [-1 * width, width * 3.5]))
        yVal = int(np.interp(lmlist[8][1], [height / 4, height], [0, height * 3.5]))

        # Gesture to start dragging (e.g., closed fist)
        if fingers == [0, 0, 0, 0, 0]:  # Closed fist gesture
            if not dragging:  # Start dragging if not already dragging
                pyautogui.mouseDown(x=xVal, y=yVal)  # Press down the mouse button
                dragging = True  # Set dragging state to true
                
        elif fingers == [1, 1, 1, 1, 1]:  # Open hand gesture to stop dragging
            if dragging:  # Stop dragging if already dragging
                pyautogui.mouseUp()  # Release the mouse button
                dragging = False  # Reset dragging state
        
        # If dragging, move the mouse
        if dragging:
            pyautogui.moveTo(xVal, yVal)  # Continuously move to the hand position
            
        else:
            # Gesture to move cursor (e.g., index finger up)
            if fingers == [0, 1, 0, 0, 0]:  # Index finger up
                pyautogui.moveTo(xVal, yVal)  # Move cursor
                
            elif fingers == [1, 1, 0, 0, 0]:  # Gesture for left click, Index finger and thumbs up
                pyautogui.click()
                time.sleep(0.7)
                
            elif fingers == [0, 1, 1, 0, 0]:  # Gesture for right click, 2 fingers up
                pyautogui.rightClick()
                time.sleep(0.7)
                
            elif fingers == [0, 1, 1, 1, 0]:  # Gesture for double click, 3 fingers up
                pyautogui.doubleClick()
                time.sleep(0.5)
    
    # show the image captured
    img = cv2.resize(img, (width, height))
    cv2.imshow('Slides', img)
    
    key = cv2.waitKey(1)
    if key == ord('q'):  # press q key to quit
        w.append(lmlist)
        cv2.destroyAllWindows()
        break
